In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

</br>

# Introduction

This problem is one of the classic machine learning problem, with a multiple datasets containing information about tennis matches, players and tournements, we try to predict the winner player in the match given the minimum information.</br>
</br>
In the following notebook I will work on different ML models to answer this problem, and at the end I will present the best model with the best accuracy score.</br>
</br>
NB : I don't have the result for the test data, my objective with this exercice is to show my technical capacities wiin ML and data analisys. During the implementation of each model I will present briefly its theory also.</br>
<font size="2.8" color="#FF4500" >Hope you enjoy it</font> 

# Environement set-up 

In [2]:
# Import libraries
import numpy as np
import sklearn
import pandas as pd
from pandas.plotting import scatter_matrix
import xgboost as xgb
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model.stochastic_gradient import SGDClassifier
%matplotlib inline

In [11]:
# Import data files
#data_train     = pd.read_csv('train.csv',index_col=0)
data_stats     = pd.read_csv('stats.csv',index_col=0)
data_test      = pd.read_csv('test.csv')
data_tour      = pd.read_csv('tour.csv')
data_players   = pd.read_csv('players.csv')
data_playerate = pd.read_csv('player_rates.csv',index_col=0)

In [ ]:
# Import test file
data_test      = pd.read_csv('test.csv')

# Data discovery 

In [ ]:
data_train.head()

Data train is the table containing examples to train the model, it could be modified to add new features to get better result. That what I will discover in the next line of code :D

ID1_G = player1 index (winner)</br>
ID2_G = player2 index (loser)</br>
ID_T_G = tournement index </br>
ID_R_G = round </br>
RESULT_G = match_result</br>
DATE_G = match date

</br>

In [ ]:
data_stats.head()

</br>

Is the table containing the most information on each match two players had 

ID1 : L’identifiant du joueur qui a gagné le match</br>
ID2 : L’identifiant du joueur qui a perdu le match</br>
ID_T : L’identifiant du tournoi (voir tours_atp)</br>
ID_R : L’identfiant du extit{round} dans le tournoi </br>
FS_1 : Nombre de premiers services réussis par joueur 1</br>
FS_OF1 : Nombre de premiers services par joueur 1</br>
ACES_1 : Aces du joueur 1</br>
DF_1 : Double Fautes commis par joueur 1</br>
UE_1 : Unforced errors</br>
W1S_1: Nombre de points gagnés sur premier service</br>
W1SOF_1: Nombre de points joués sur premier service</br>
W2S_1: Nombre de points gagnés sur second service</br>
W2SOF_1: Nombre de points joués sur second service</br>
WIS_1 : Nombre de points gagnés en tout</br>
BP_1 : Nombre de balles de break gagnées</br>
BPOF_1 : Nombre de balles de break obtenues</br>
RPW_1 : Nombre de points gagnés</br>
RPW_OF1 : Nombre de points jouées</br>
Same features for player 2

</br>

In [ ]:
data_players.head()

The table containing information on each player

</br>

In [ ]:
data_playerate.head()

Table containing the ranking of each player per year

</br>

In [ ]:
data_tour.head()

Table containing information about each tournement

## Data pre-processing 

### Missing values

In [ ]:
data_train.isnull().sum()/data_train.shape[0] * 100

In [ ]:
data_players.isnull().sum()/data_players.shape[0] * 100

As it is shown for the table players most features are Null, and we could not use any of the technique to complete them

In [ ]:
data_playerate.isnull().sum()/data_playerate.shape[0] * 100

In [ ]:
data_stats.isnull().sum()/data_stats.shape[0] * 100

Some columns contains lot of missing values, I will drop them since I could not replace them with certain values (mean, median)</br>
I should mention that during the next step, "data preprocessing", I will complete the missing values, drop certain columns and make opperations between columns having a certain correlation

In [ ]:
data_tour.isnull().sum()/data_tour.shape[0] * 100

## Filling missing values 

The first data preprocessing is to combine columns with hight correlation, without any calculations, we can understaind that for the number of winning point during the first serve is correlated to the number of first serve W1S_1 to W1SOF_1 (same for player2), so I will combine this two columns in one column reflecting the rate of winning first serve </br>
The same idea goes for many columns : </br>
W1S_1 and W1SOF_1</br>
W2S_1 and W2SOF_1</br>
FS_1 and FS_OF_1</br>
BP_1 and BPOF_1</br>
RPW_1 and RPW_OF1</br>
and the same thing goes for player 2 </br>
Then I will drop thes columns

In [ ]:
tuples = [('W1SR_1', 'W1S_1', 'W1SOF_1'),
          ('W2SR_1', 'W2S_1', 'W2SOF_1'),
          ('FSR_1', 'FS_1', 'FSOF_1'),
          ('BPR_1', 'BP_1', 'BPOF_1'),
          ('RPWR_1', 'RPW_1', 'RPWOF_1'),
          ('W1SR_2', 'W1S_2', 'W1SOF_2'),
          ('W2SR_2', 'W2S_2', 'W2SOF_2'),
          ('FSR_2', 'FS_2', 'FSOF_2'),
          ('BPR_2', 'BP_2', 'BPOF_2'),
          ('RPWR_2', 'RPW_2', 'RPWOF_2'),
]
for tup in tuples:
    data_stats[tup[0]] = data_stats[tup[1]] / data_stats[tup[2]]

In [ ]:
data_stats.drop(['W1S_1', 'W1SOF_1', 'W2S_1', 'W2SOF_1', 'FS_1', 'FSOF_1', 'BP_1', 'BPOF_1', 'RPW_1', 'RPWOF_1', 
                 'W1S_2', 'W1SOF_2', 'W2S_2', 'W2SOF_2', 'FS_2', 'FSOF_2', 'BP_2', 'BPOF_2', 'RPW_2', 'RPWOF_2'],
               inplace = True, axis = 1)

In [ ]:
data_stats.drop(['UE_1', 'WIS_1', 'NA_1', 'NAOF_1', 'A1S_1', 'A2S_1', 'FAST_1',
                 'UE_2', 'WIS_2', 'NA_2', 'NAOF_2', 'A1S_2', 'A2S_2', 'FAST_2'],
               inplace = True, axis = 1)

In [ ]:
data_stats.isnull().sum()/data_stats.shape[0] * 100

## Data format

In [12]:
# Formattage des colonnes dates
def date_tras(df,name):
    df[name] = pd.to_datetime(df[name])

In [13]:
date_tras(data_train,'DATE_G')
date_tras(data_stats,'MT')
date_tras(data_playerate,'DATE_R')

# Some stats 

In [ ]:
print("The number of training examples: {} \nThe number of training features: {}".format(data_train.shape[0],data_train.shape[1]))

In [ ]:
print("Number of distinct players : {} \nThe number of training features: {}".format(data_players.NAME_P.unique().shape[0],data_players.shape[1]))

In [ ]:
print("Number of distinct tournement : {} \nThe number of training features: {}".format(data_tour.NAME_T.unique().shape[0],data_tour.shape[1]))

### The top ten winners player

In [ ]:
top10player = {}
top10player_list = data_stats.ID1.value_counts().head(10)
for player_idx in top10player_list.index:
    top10player[data_players.NAME_P[data_players.ID_P == player_idx].iloc[0]] = top10player_list.loc[player_idx]

In [ ]:
plt.figure(figsize=(15,10))
plt.bar(range(len(top10player)), top10player.values(), align='center', color = 'green',alpha= 0.3)
plt.xticks(range(len(top10player)), list(top10player.keys()))
plt.title("The top 10 player")
plt.show()

#### Correlation matrix for top 10 players

In [ ]:
data_stats['top_ten'] = data_stats.ID1.apply(lambda x: x in top10player_list.index)

In [ ]:
data_stats_top_ten = data_stats[data_stats.top_ten == True]

In [ ]:
corr = data_stats_top_ten[['ID1','DF_1', 'TPW_1','ACES_1','W1SR_1', 'W2SR_1', 'FSR_1', 'BPR_1', 'RPWR_1','ID_R']].corr()

In [ ]:
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In the graph there is some logical results and other that need to be reviewd, the winning rate is positively correlated to the number of Double faults is so strange, but for other metrics they are logicaly correlated :D.</br>
We notice that the winning rate is so correlated with number of aces, which means that the player wins easely the point and the match is not so challenging.</br>
Same thing goes for the other features

### top 3 Favorit tournements for the 10 top players

In [ ]:
player_tournement = {}
for player_idx in top10player_list.index:
    player_name = data_players.NAME_P[data_players.ID_P == player_idx].iloc[0]
    player_tournement[player_name] = data_stats.ID_T[data_stats.ID1 == player_idx].value_counts().head(3)
    player_tournement[player_name].index = list(data_tour.NAME_T.iloc[player_tournement[player_name].index])

In [ ]:
for player in player_tournement:
    print("Top 3 tournement for '{}'".format(player))
    print(player_tournement[player])
    print("\n")

It is so clear that this dataset is so old as these players won lot of well known tournements </br> <font color="red">'NADAL' and 'Kazakhstan' so weird !!!!!</font>

## Feature preparation

#### functions definition

In [ ]:
# Récuperer les statistique des derniers n matches entre (player1_id,player2_id)
def last_n_match(player1_id,player2_id,n,dt_curr):
    a = np.bitwise_and((data_stats.ID1 == player1_id).values,(data_stats.ID2 == player2_id).values)    
    b = np.bitwise_and((data_stats.ID1 == player2_id) , (data_stats.ID2 == player1_id))
    x = np.bitwise_or(a,b)
    
    # List des matches entre(player1_id,player2_id)
    players_matches = list(data_stats[x.values].index)
    
    # Si on a plus que n matches, on prend les n derniers
    if (n <= len(players_matches)):
        matches = data_stats.loc[players_matches]
        matches = matches.sort_values('MT',ascending=False)
        matches = matches[matches.MT < dt_curr]
        return matches.head(n)
    
    # Sinon on prend tous les matches ou retourne NONE
    elif (0<len(players_matches) and len(players_matches)<n):
        matches = data_stats.loc[players_matches]
        matches = matches.sort_values('MT',ascending=False)
        matches = matches[matches.MT <= dt_curr]
        return matches.head(len(players_matches))
    else:
        return None

In [ ]:
# Récupere les résultats
def get_info():
    k = 0
    for i in list(data_train.index):
        print("chargement : {:.2f} %".format(k / len(list(data_train.index)) * 100))
        k +=1
        
        info_i = last_n_match(data_train.ID1_G.loc[i],data_train.ID2_G.loc[i],2,data_train.DATE_G.loc[i])
        
        data_train.loc[i,'PL1_1'] = 1
        # le nombre de matche joueur 1 a gangné
        oc1 = [0]*2
        
        # le nombre de matche joueur 2 a gangné
        oc2 = [0]*2
        
        # Compléter la table des startistiques
        if info_i is not None:
            k1 = 0
            k2 = 0
            for j in list(info_i.ID1.index):
                if(info_i.ID1.loc[j] == data_train.ID1_G.loc[i]):
                    oc1[k1] = 1
                    k1+=1
                else:
                    oc2[k2] = 1
                    k2 +=1
            
            vec = pd.concat([pd.Series(oc1),pd.Series(oc2)],axis=0,ignore_index=True)
            data_train.loc[i,'PL1_2'] = vec[0]
            data_train.loc[i,'PL1_3'] = vec[1]
            data_train.loc[i,'PL2_2'] = vec[2]
            data_train.loc[i,'PL2_3'] = vec[3]

In [9]:
# Récuperer le meilleur classement du jour avant la date du matche
def get_rate(player_id,dt_curr):   
    player = data_playerate[data_playerate.ID_P_R == player_id]
    if (len(player[player.DATE_R <= dt_curr]) != 0):
        return max(player.POINT_R)
    elif (len(player)!=0):
        return max(player.POINT_R)
    else:
        return 0

In [ ]:
# Récuper les différence entre les points gangés dans les n dernier matche entre joueur1 et jouer2

## W1S_j : Nombre de points gagnés sur premier service
## W2S_j : Nombre de points gagnés sur second service
## BP    : Nombre de balles de break gagnées

def get_point():
    k = 0
    for i in list(data_train.index):
        print("chargement : {:.2f} %".format(k / len(data_train.ID1_G) * 100))
        k +=1
        info_i = last_n_match(data_train.ID1_G.loc[i],data_train.ID2_G.loc[i],3,data_train.DATE_G.loc[i])
        
        if info_i is not None:
            k1 = 0
            k2 = 0
            for j in list(info_i.ID1.index):
                if(info_i.ID1.loc[j] == data_train.ID1_G.loc[i]):
                    k1 = info_i.W1SR_1.loc[j] + info_i.W2SR_1.loc[j] + info_i.BPR_1.loc[j]
                    k2 = info_i.W1SR_2.loc[j] + info_i.W2SR_2.loc[j] + info_i.BPR_2.loc[j]
                else:
                    k2 = info_i.W1SR_1.loc[j] + info_i.W2SR_1.loc[j] + info_i.BPR_1.loc[j]
                    k1 = info_i.W1SR_2.loc[j] + info_i.W2SR_2.loc[j] + info_i.BPR_2.loc[j]
            data_train.DIFF_pts.loc[i] = k1 - k2

In [ ]:
def ajt(df):
    # Resultats des 3 deriniers matches
    df['PL1_1'] = [0]*df.ID1_G
    df['PL1_2'] = [0]*df.ID1_G
    df['PL1_3'] = [0]*df.ID1_G
    df['PL2_1'] = [0]*df.ID1_G
    df['PL2_2'] = [0]*df.ID1_G
    df['PL2_3'] = [0]*df.ID1_G



    # Difference entre les points gagnés par les deux joueurs dans les 3 derniers matches
    df['DIFF_pts'] = [0]*df.shape[0]

In [ ]:
ajt(data_train)

In [ ]:
data_train_bck = data_train

In [ ]:
get_info()

In [ ]:
get_point()

In [ ]:
data_train.to_csv('train_data_formated.csv')

In [6]:
data_train = pd.read_csv('train_data_formated.csv',index_col=0)

In [7]:
data_train.head(5)

,ID1_G,ID2_G,ID_T_G,ID_R_G,RESULT_G,DATE_G,PL1_1,PL1_2,PL1_3,PL2_1,...,R8,R9,R10,R11,R12,R13,R14,R15,R16,R17
0,17613,18854,8887,4,7-6(8) 3-6 10-7,2012-01-31,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25192,25191,8888,4,6-3 6-2,2012-01-31,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,918,10828,8957,4,6-1 6-2,2012-01-31,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,3316,2379,8957,4,6-4 7-6(5),2012-01-31,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7869,6277,8957,4,6-4 6-4,2012-01-31,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
data_train['label'] = [1] * data_train.shape[0]
from random import randint
rn = []
rnn = [randint(0,data_train.shape[0]-1) for i in range(int(data_train.shape[0]/3))]
for i in rnn:
    rn.append(data_train.index[i])
    
# Permutation des données
for i in rn:
    data_train.loc[i,'PL1_1'],data_train.loc[i,'PL2_1'] = data_train.loc[i,'PL2_1'],data_train.loc[i,'PL1_1']
    data_train.loc[i,'PL1_2'],data_train.loc[i,'PL2_2'] = data_train.loc[i,'PL2_2'],data_train.loc[i,'PL1_2']
    data_train.loc[i,'PL1_3'],data_train.loc[i,'PL2_2'] = data_train.loc[i,'PL2_3'],data_train.loc[i,'PL1_3']
    
    data_train.loc[i,'DIFF_pts'] = -1*data_train.loc[i,'DIFF_pts']     
    
    # joueur 2 gangant => label = 0
    data_train.loc[i,'label'] = 0

In [22]:
data_train.to_csv('train_data_formated_1.csv')

# Models

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [63]:
data_train.drop('ID1_G',axis=1, inplace=True)
data_train.drop('ID2_G',axis=1, inplace=True)
data_train.drop('ID_T_G',axis=1, inplace=True)
data_train.drop('ID_R_G',axis=1, inplace=True)
data_train.drop('DATE_G',axis=1, inplace=True)
data_train.drop('RESULT_G',axis=1, inplace=True)
data_train.drop(['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8', 'R9', 'R10', 'R11', 'R12',
       'R13', 'R14', 'R15', 'R16', 'R17'],axis = 1, inplace =True)

In [65]:
X = data_train.drop(['label'],1)
X = X.fillna(0)
Y = data_train.label

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size = 0.3,
                                                    random_state = 2,
                                                    stratify = Y)

## XgBoost

### Presentation

XgBoost is one the esemble Machine learning models, there is problems in machine learning where learning a single model is not suficent, thus combining multiple ML models is not a bad idea.
Briefly esemble methodes consist on combining some machine learning weak models following some a specific architecture (bagging, boosting) to come up with a better model at the end.
XgBoost is a boosting model, one of the most used models in Kaggle competition for its powerful capacity to learn and also its scalability to run on multiple computers (team members) also the parallel computing possibility that boosting models offer.
During the Boosting learning multiple trees are built sequentially such that a new tree reduce error made by the previous one.
let M0 be the first model, it has an error (y - M0(x)), the next model will  learn on (y - M0(x)) and come up with the model M1, and this process goes on ...

 <img src="e7MIgXk.png" alt="Girl in a jacket" width="25%" > 
<br>
<br>source : https://www.analyticsvidhya.com/blog/2018/09/an-end-to-end-guide-to-understand-the-math-behind-xgboost/
<br><strong>Regularization</strong>: XGBoost has an option to penalize complex models through both L1 and L2 regularization. Regularization helps in preventing overfitting
<br><strong>Handling sparse data</strong>: Missing values or data processing steps like one-hot encoding make data sparse. XGBoost incorporates a sparsity-aware split finding algorithm to handle different types of sparsity patterns in the data
<br><strong>Weighted quantile sketch</strong>: Most existing tree based algorithms can find the split points when the data points are of equal weights (using quantile sketch algorithm). However, they are not equipped to handle weighted data. XGBoost has a distributed weighted quantile sketch algorithm to effectively handle weighted data
<br><strong>Block structure for parallel learning</strong>: For faster computing, XGBoost can make use of multiple cores on the CPU. This is possible because of a block structure in its system design. Data is sorted and stored in in-memory units called blocks. Unlike other algorithms, this enables the data layout to be reused by subsequent iterations, instead of computing it again. This feature also serves useful for steps like split finding and column sub-sampling
<br><strong>Cache awareness</strong>: In XGBoost, non-continuous memory access is required to get the gradient statistics by row index. Hence, XGBoost has been designed to make optimal use of hardware. This is done by allocating internal buffers in each thread, where the gradient statistics can be stored
<br><strong>Out-of-core computing</strong>: This feature optimizes the available disk space and maximizes its usage when handling huge datasets that do not fit into memory



### Parameters to tune

Source : https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
<br>For the XgBoost model there is lot of parameters to tune and it takes much time to train, thus I will choose some of these parametres that I see are important.
<br>
<br>
<strong>max_depth (int) </strong>– Maximum tree depth for base learners.
<br>
<strong>learning_rate (float) </strong>– Boosting learning rate (xgb’s “eta”)
<br>
<strong>n_estimators (int) </strong>– Number of trees to fit.
<br>
<strong>booster (string) </strong>– Specify which booster to use: gbtree, gblinear or dart.
<br>
<strong>n_jobs (int) </strong>– Number of parallel threads used to run xgboost. (replaces nthread)
<br>
<strong>gamma (float) </strong>– Minimum loss reduction required to make a further partition on a leaf node of the tree.
<br>
<strong>max_delta_step (int) </strong>– Maximum delta step we allow each tree’s weight estimation to be.
<br>
<strong>subsample (float) </strong>– Subsample ratio of the training instance.
<br>
<strong>colsample_bytree (float) </strong>– Subsample ratio of columns when constructing each tree.
<br>
<strong>reg_alpha (float (xgb's alpha)) </strong>– L1 regularization term on weights
<br>
<strong>reg_lambda (float (xgb's lambda)) </strong>– L2 regularization term on weights
<br>
<strong>scale_pos_weight (float) </strong>– Balancing of positive and negative weights.

## Logistic Regression

### Presentation

Logistic regression is a classification machine learning model, make use of the simlicty of linear model, LR introduces the sigmoid function to control the output so that the result will take values in {0,1}. Note that for multiple label LR problem multiple binary LR models are learnt as it is the case for sk-learn library. 
 <img src="LogReg_1.png" alt="Girl in a jacket" width="35%" > 
<br>

### Parameters to tune

Source : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
<br>
<br><strong>penalty </strong> : 'l1','l2’, ‘elasticnet’ or ‘none’, optional (default=’l2’)
<br><strong>dual</strong> : bool, optional (default=False)
<br><strong>tol</strong> : float, optional (default=1e-4)
<br><strong>C</strong> : float, optional (default=1.0)
<br><strong>fit_intercept</strong> : bool, optional (default=True)
<br><strong>max_iter</strong> : int, optional (default=100)
<br><strong>multi_class</strong> : str, {‘ovr’, ‘multinomial’, ‘auto’}, optional (default=’ovr’)

In [24]:
from sklearn.linear_model import LogisticRegression

In [74]:
parameters = { 'C' : [0.01,0.1,1,10],
               'tol' : [0.001,0.01,0.1],
              'solver' : ['lbfgs', 'liblinear']
             }  

clf_x = LogisticRegression()

f1_scorer = make_scorer(f1_score,pos_label=1)

grid_obj = GridSearchCV(clf_x,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

grid_obj = grid_obj.fit(X_train,y_train)

clf_x = grid_obj.best_estimator_
print (clf_x)


y = clf_x.predict(X_train)
print(f1_score(y_train, y, pos_label=1))


y = clf_x.predict(X_test)
print(f1_score(y_test, y, pos_label=1))

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.001, verbose=0, warm_start=False)
0.9804360573591343
0.979402901612352


We got a high f1-score with this model, which is a little bit weird to get this value on real data. Let's analyse the coefficient we get.

In [75]:
clf_x.coef_

array([[ 4.46144315,  4.63390253,  0.96598404, -8.17729698,  3.53709297,
        -0.6893501 , -1.76888051]])

As it is shown the label depends on the first features on the table, such information can not be trustful. I should think of another way to present my data

In [100]:
#Sum the values of feature PLi_j in new feature P_i
data_train['p_1'] = data_train.apply(lambda x:(x.PL1_2+x.PL1_1+x.PL1_3)/3 ,axis=1)
data_train['p_2'] = data_train.apply(lambda x:(x['PL2_1']+x['PL2_2']+x['PL2_3'])/3,axis = 1)

In [105]:
corr = data_train(['p_1','p_2','DIFF_pts','label']).corr()

TypeError: 'DataFrame' object is not callable

In [103]:
corr

,PL1_1,PL1_2,PL1_3,PL2_1,PL2_2,PL2_3,DIFF_pts,label,p_1,p_2
PL1_1,1.000000,0.680801,0.178802,-1.000000,-0.030647,-0.000217,0.495483,0.902704,0.850293,-0.759420
PL1_2,0.680801,1.000000,0.239083,-0.680801,-0.043323,-0.106583,0.450713,0.749521,0.882579,-0.550651
PL1_3,0.178802,0.239083,1.000000,-0.178802,-0.132796,0.060550,0.126230,0.198430,0.533158,-0.198165
PL2_1,-1.000000,-0.680801,-0.178802,1.000000,0.030647,0.000217,-0.495483,-0.902704,-0.850293,0.759420
PL2_2,-0.030647,-0.043323,-0.132796,0.030647,1.000000,0.211817,-0.148151,0.001944,-0.079648,0.645570
PL2_3,-0.000217,-0.106583,0.060550,0.000217,0.211817,1.000000,-0.005317,0.000851,-0.032426,0.316656
DIFF_pts,0.495483,0.450713,0.126230,-0.495483,-0.148151,-0.005317,1.000000,0.447875,0.493002,-0.456532
label,0.902704,0.749521,0.198430,-0.902704,0.001944,0.000851,0.447875,1.000000,0.846464,-0.668100
p_1,0.850293,0.882579,0.533158,-0.850293,-0.079648,-0.032426,0.493002,0.846464,1.000000,-0.683144
p_2,-0.759420,-0.550651,-0.198165,0.759420,0.645570,0.316656,-0.456532,-0.668100,-0.683144,1.000000


The value of the label depends on the results 

## SVM

### Presentation

In [76]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

parameters = { 'learning_rate' : [0.001,0.0001],
               'n_estimators' : [100,120,140],
               'max_depth': [4,6],
               'min_child_weight': [3],
               'gamma':[0.4,0.8],
               'subsample' : [0.2,0.5,0.8],
               'colsample_bytree' : [0.5,0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             }  

clf = xgb.XGBClassifier()

f1_scorer = make_scorer(f1_score,pos_label=1)

grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

grid_obj = grid_obj.fit(X_train,y_train)

clf_svm = grid_obj.best_estimator_
print (clf)


y = clf.predict(X_train)
print(f1_score(y_train, y, pos_label=1))


y = clf.predict(X_test)
print(f1_score(y_test, y, pos_label=1))

KeyboardInterrupt: 